In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

<div class="alert alert-block alert-info">
<b>To do</b>:

 - Encoding
 - Algo
    
Subsidiary questions: Product Life cycel status (RaMPDOWN)
</div>

# **1. Loading and Preprocessing**

## *Train dataset*

In [2]:
X = pd.read_csv('datasets_hi4/train-data.csv', sep=';')
X.drop_duplicates(inplace=True, ignore_index=True)

In [3]:
country_mapping = {'DE': 'Germany', 'CN': 'China', 'GB': 'United Kingdom', 'AU': 'Australia', 'ES': 'Spain',
                   'NL': 'Netherlands', 'US': 'United States', 'DK': 'Denmark', 'BE': 'Belgium', 'FR': 'France',
                   'IT': 'Italy', 'PL': 'Poland', 'SE': 'Sweden', 'TW': 'Taiwan', 'JP': 'Japan', 'HK': 'Hong Kong',
                   'KR': 'South Korea', 'PH': 'Philippines', 'MY': 'Malaysia', 'SG': 'Singapore', 'TH': 'Thailand',
                   'Id': 'Indonesia', 'FJ': 'Fiji', 'VN': 'Vietnam', 'BN': 'Brunei', 'NZ': 'New Zealand',
                   'MM': 'Myanmar', 'KH': 'Cambodia', 'MN': 'Mongolia', 'LA': 'Laos', 'PG': 'Papua New Guinea'}

X['Country'] = X['Country'].map(country_mapping)

In [4]:
X['Reference proxy'] = X['Reference proxy'].apply(lambda x: x.replace('reference-', ''))
X['Product  Line proxy'] = X['Product  Line proxy'].apply(lambda x: x.replace('Product Line-', ''))
X['Division proxy'] = X['Division proxy'].apply(lambda x: x.replace('Division-', ''))
X['Customer Persona proxy'] = X['Customer Persona proxy'].apply(lambda x: x.replace('Customer Segmentation-', ''))
X['Strategic Product Family proxy'] = X['Strategic Product Family proxy'].apply(lambda x: x.replace('Strategic Product Family-', ''))

In [5]:
X[['Quarter', 'Year']] = X.Date.str.split(expand=True)
X['Quarter'] = X['Quarter'].map({'jan-apr': 1, 'may-jul': 2, 'may-aug': 2, 'sep-dec': 3})
X['Quarter_cumuled'] = (X['Quarter']-2) + (X['Year'].astype(int)-2020)*3

In [6]:
X['Site'] = X['Site'].map({'OOS_DRP': 'Unknown_Unknown_OOS_DRP'}).fillna(X['Site'])
site_det = X['Site'].str.split('_', expand=True)
site_det[2] = pd.Series(np.where(site_det[3].notna(), site_det[2]+' '+ site_det[3], site_det[2]))
X[['Site_country', 'Site_type', 'Site_complementary']] = site_det[[0, 1, 2]]
X.drop('Site', axis=1, inplace=True)

In [7]:
X['Month 1'].fillna(X['Month 2'], inplace=True)
X['Product Life cycel status'] = X['Product Life cycel status'].map({'NPI': 3, 'EOL': 0, 'RaMPDOWN': 1}).fillna(2)

## *Other datasets*

In [8]:
df_gscpi = pd.read_csv('datasets_hi4/extra-dataset/GSCPI_data.csv')

In [9]:
df_lpi = pd.read_csv('datasets_hi4/extra-dataset/LPIextend.csv')

In [10]:
df_worldEconomy = pd.read_csv('datasets_hi4/extra-dataset/worldbank_economic_data.csv')
df_worldEconomy['Year'] = df_worldEconomy['Year'].astype(str)

In [11]:
df_worldInfla = pd.read_csv('datasets_hi4/extra-dataset/worldbank_inflation_data.csv')
df_worldInfla = pd.merge(df_worldInfla, df_gscpi, on='Year-Month', how='left')

In [12]:
def get_Date(x) : 
    year = x.split('-')[0]
    month = int(x.split('-')[1])

    if year == '2023' :
        if month > 4 : 
            return 'may-jul 2023'
        else : 
            return 'jan-apr 2023'
    
    if month <= 4 : 
        return 'jan-apr ' + year
    elif month <=8 : 
        return 'may-aug ' + year
    else : 
        return 'sep-dec ' + year

In [13]:
df_worldInfla['Date'] = df_worldInfla['Year-Month'].apply(get_Date)

In [14]:
result_df1 = df_worldInfla.groupby(['Country', 'Date'])['Energy Price Index'].apply(lambda x: x.tolist()).reset_index(name='Energy_Price_Index_List')
result_df2 = df_worldInfla.groupby(['Country', 'Date'])['Headline Consumer Price Index'].apply(lambda x: x.tolist()).reset_index(name='Headline_Consumer_Index_List')
result_df3 = df_worldInfla.groupby(['Country', 'Date'])['GSCPI'].apply(lambda x: x.tolist()).reset_index(name='GSCPI_List')

In [15]:
result_df1_expanded = result_df1['Energy_Price_Index_List'].apply(pd.Series)
result_df1_expanded = result_df1_expanded.rename(columns=lambda x: f'Energy_Price_Index_{x + 1}')

result_df2_expanded = result_df2['Headline_Consumer_Index_List'].apply(pd.Series)
result_df2_expanded = result_df2_expanded.rename(columns=lambda x: f'Headline_Consumer_Index_{x + 1}')

result_df3_expanded = result_df3['GSCPI_List'].apply(pd.Series)
result_df3_expanded = result_df3_expanded.rename(columns=lambda x: f'GSCPI_{x + 1}')

In [16]:
df_final1 = pd.concat([result_df1, result_df1_expanded], axis=1)
df_final2 = pd.concat([result_df2, result_df2_expanded], axis=1)
df_final3 = pd.concat([result_df3, result_df3_expanded], axis=1)

In [17]:
df_final = pd.merge(df_final1, df_final2, on = ['Country', 'Date'], how='inner')
df_final = pd.merge(df_final, df_final3, on = ['Country', 'Date'], how='inner')

In [18]:
df_final = df_final.drop(['Headline_Consumer_Index_List', 'Energy_Price_Index_List', 'GSCPI_List'], axis=1)

In [21]:
X = pd.merge(X, df_final, on=['Country', 'Date'], how='left')
X = pd.merge(X, df_worldEconomy, on=['Country', 'Year'], how='left')
X = pd.merge(X, df_lpi, on=['Country'], how='left')
X.drop('Date', axis=1, inplace=True)

In [27]:
X.head()

,index,id_product,Region,Country,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Month 1,Month 2,Month 3,Month 4,Quarter,Year,Quarter_cumuled,Site_country,Site_type,Site_complementary,Energy_Price_Index_1,Energy_Price_Index_2,Energy_Price_Index_3,Energy_Price_Index_4,Headline_Consumer_Index_1,Headline_Consumer_Index_2,Headline_Consumer_Index_3,Headline_Consumer_Index_4,GSCPI_1,GSCPI_2,GSCPI_3,GSCPI_4,"Agriculture, forestry, and fishing, value added (annual % growth)",Exports of goods and services (annual % growth),Final consumption expenditure (annual % growth),GDP (current US$),Gross capital formation (annual % growth),Imports of goods and services (annual % growth),"Industry (including construction), value added (annual % growth)","Manufacturing, value added (annual % growth)","Services, value added (annual % growth)",Unnamed: 0,ID,population (2023),area,landAreaKm,unMember,netChange,growthRate,worldPercentage,density,densityMi,rank,LPI Grouped Rank,Customs Score,Customs Grouped Rank,Infrastructure Score,Infrastructure Grouped Rank,International Shipments Score,International Shipments Grouped Rank,Logistics Competence and Quality Score,Logistics Competence and Quality Grouped Rank,Timeliness Score,Timeliness Grouped Rank,Tracking and Tracing Score,Tracking and Tracing Grouped Rank
0,645874,156160,EUROPE,Germany,Europe Operations,DACH,Germany,13523,4,3,3,12,2.0,0,0,0,0,2,2021,3,NL,DC,Venray,107.106,108.0580,109.501,110.149,108.170000,108.486000,109.013000,109.118000,2.981772,2.694348,2.922301,3.232759,1.61,9.71,1.44,4.259935e+12,3.50,9.00,3.61,5.07,2.27,3,3,8.329463e+07,357114.0,349390.0,True,-0.0012,-0.0009,0.0104,238.4002,617.4564,19.0,3.0,3.9,7.0,4.3,3.0,3.7,8.0,4.2,3.0,4.1,10.0,4.2,3.0
1,469488,224631,CHINA,China,China Operations,China & HK,China,12778,4,3,14,9,2.0,0,1,0,0,1,2021,2,CN,DC,Shanghai,100.800,100.9000,100.900,100.900,129.344851,130.113998,129.473043,129.088469,1.329892,1.908749,2.199072,2.682921,7.07,NaN,9.08,1.782046e+13,3.87,NaN,8.67,NaN,8.49,17,17,1.425671e+09,9706961.0,9424702.9,True,-0.0113,-0.0002,0.1781,151.2696,391.7884,2.0,19.0,3.3,31.0,4.0,14.0,3.6,14.0,3.8,20.0,3.7,30.0,3.8,23.0
2,348904,104047,EUROPE,United Kingdom,Europe Operations,UK and Ireland,United Kingdom,3513,3,2,23,7,2.0,0,2,0,0,1,2021,2,NL,DC,HLD,103.600,105.1000,106.500,112.000,109.300000,109.400000,109.700000,110.400000,1.329892,1.908749,2.199072,2.682921,6.19,2.24,7.76,3.122480e+12,13.98,6.19,9.29,10.08,7.53,22,22,6.773680e+07,242900.0,241930.0,True,0.0072,0.0034,0.0085,279.9851,725.1615,21.0,19.0,3.5,22.0,3.7,25.0,3.5,22.0,3.7,28.0,3.7,30.0,4.0,16.0
3,1725822,11823,EAJP,Australia,International Operations,Pacific,Australia,672,1,1,11,3,2.0,0,0,0,0,1,2023,8,AU,DC,Perth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,16,2.643911e+07,7692024.0,7692020.0,True,0.0083,0.0100,0.0033,3.4372,8.9024,55.0,19.0,3.7,14.0,4.1,9.0,3.1,47.0,3.9,14.0,3.6,35.0,4.1,11.0
4,404781,159924,EUROPE,Spain,Europe Operations,Iberia,Spain,3496,3,2,14,7,2.0,0,0,0,0,1,2021,2,NL,DC,HLD,101.619,96.4762,102.303,106.463,104.500000,103.900000,104.900000,106.100000,1.329892,1.908749,2.199072,2.682921,2.07,14.38,5.09,1.427381e+12,5.86,13.92,4.03,8.91,5.95,14,14,4.751963e+07,505992.0,499556.6,True,-0.0014,-0.0008,0.0059,95.1236,246.3702,32.0,13.0,3.6,20.0,3.8,19.0,3.7,8.0,3.9,14.0,4.2,4.0,4.1,11.0


In [28]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1762970 entries, 0 to 1762969
Data columns (total 69 columns):
 #   Column                                                             Dtype  
---  ------                                                             -----  
 0   index                                                              int64  
 1   id_product                                                         int64  
 2   Region                                                             object 
 3   Country                                                            object 
 4   Operations                                                         object 
 5   Zone                                                               object 
 6   Cluster                                                            object 
 7   Reference proxy                                                    object 
 8   Product  Line proxy                                                object 
 9   Di

In [22]:
#string_columns = X.select_dtypes(include=['object']).columns

#for col in string_columns:
#    le = LabelEncoder()
#    X[col] = le.fit_transform(X[col])

In [23]:
#matrice_corr = np.abs(X.corr())
#masque_triangulaire = np.tril(np.ones(matrice_corr.shape), k=-1).astype(bool)
#matrice_corr_triangulaire = matrice_corr.where(masque_triangulaire)

In [24]:
#plt.figure(figsize=(8, 6))
#sns.heatmap(matrice_corr_triangulaire, cmap='coolwarm', mask=~masque_triangulaire)
#plt.title('Matrice de corrélation en valeurs absolues (sous forme triangulaire)')
#plt.show()